In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass



CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


In [14]:
###
### Common ###
###

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()

###
### Torch ###
###

_IMPORT_('import torch')
_IMPORT_('import torch.nn as nn')
_IMPORT_('import torch.nn.functional as F')
_IMPORT_('import torch.optim as O')
_IMPORT_('from torchvision import models as M')
_IMPORT_('from torchvision import transforms as T')
_IMPORT_('from torch.utils.data import Dataset, DataLoader')

###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')

# plotly.offline.init_notebook_mode(connected=False)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_video(vidsrc, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if vidsrc.startswith('http'):
        data_url = vidsrc
    else:
        mp4 = open(vidsrc, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if image_path.startswith('http'):
        data_url = image_path
    else:
        img = open(image_path, 'rb').read()
        data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

def im_read(url, rgb=True, size=None):
    response = requests.get(url)
    if response:
        imgmat = np.frombuffer(response.content, dtype=np.uint8)
        img = cv2.imdecode(imgmat, cv2.IMREAD_COLOR)
        if rgb:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if size:
           if isinstance(size, int):
               size = (size, size)
           img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
        return img
    return None


from k12libs.utils.nb_easy import RACEURL

In [18]:
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
API_INFERENCE

'http://116.85.5.40:9119/raceai/framework/inference'

In [20]:
API_POPMSG = f'{RACEURL}/raceai/private/popmsg'
API_POPMSG 

'http://116.85.5.40:9119/raceai/private/popmsg'

In [5]:
MODEL_TASK = 'zmq.fruit.resnet18.inference'
MSGKEY = 'zmq.fruit.pl.test'
IMG_ROOT = '/raceai/data/datasets/fruit/'

In [7]:
!cat $IMG_ROOT/info.json 

{
    "num_records": 75,
    "num_classes": 5,
    "label_names": [
        "carrot",
        "banana",
        "pineapple",
        "cherry",
        "tomatoes"
    ],
    "mean": [
        0.8114,
        0.6869,
        0.5652
    ],
    "std": [
        0.1771,
        0.2393,
        0.3218
    ]
}

## 输入

In [21]:
cfgdata = '''{
    "task": "%s",
    "cfg": {
        "pigeon": {
            "msgkey": "%s",
            "user": "1",
            "uuid": "100"
        },
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": [
                    {"image_path": "tomatoes/tomatoes4.jpg", "image_id": 1},
                    {"image_path": "cherry/cherry3.jpg", "image_id": 2},
                    {"image_path": "banana/banana3.jpg", "image_id": 3}
                ],
                "dataset": {
                    "class_name": "raceai.data.PredictListImageDataset",
                    "params": {
                        "data_prefix": "%s",
                        "input_size": 224
                    }
                },
                "sample": {
                    "batch_size": 32,
                    "num_workers": 4,
                }
            }
        }
    }
}''' % (MODEL_TASK, MSGKEY, IMG_ROOT)

```json
{
    "task": "zmq.fruit.resnet18.inference",
    "cfg": {
        "pigeon": {
            "msgkey": "zmq.fruit.pl.test",
            "user": "1",
            "uuid": "100"
        },
        "data": {
            "class_name": "raceai.data.process.PathListDataLoader",
            "params": {
                "data_source": [
                    {"image_path": "tomatoes/tomatoes4.jpg", "image_id": 1},
                    {"image_path": "cherry/cherry3.jpg", "image_id": 2},
                    {"image_path": "banana/banana3.jpg", "image_id": 3}
                ],
                "dataset": {
                    "class_name": "raceai.data.PredictListImageDataset",
                    "params": {
                        "data_prefix": "/raceai/data/datasets/fruit/",
                        "input_size": 224
                    }
                },
                "sample": {
                    "batch_size": 32,
                    "num_workers": 4,
                }
            }
        }
    }
}
```

In [23]:
reqdata = eval(cfgdata)
json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
json.loads(requests.post(url=API_INFERENCE, json=reqdata).text)
time.sleep(1)

In [24]:
json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)

[{'pigeon': {'msgkey': 'zmq.fruit.pl.test', 'user': '1', 'uuid': '100'},
  'task': 'zmq.fruit.resnet18.inference',
  'errno': 0,
  'result': [{'image_path': '/raceai/data/datasets/fruit/tomatoes/tomatoes4.jpg',
    'image_id': 1,
    'probs': [0.05346506088972092,
     0.0044427961111068726,
     0.7662657499313354,
     0.006293818820267916,
     0.16953247785568237],
    'probs_sorted': {'values': [0.7662657499313354,
      0.16953247785568237,
      0.05346506088972092,
      0.006293818820267916,
      0.0044427961111068726],
     'indices': [2, 4, 0, 3, 1]}},
   {'image_path': '/raceai/data/datasets/fruit/cherry/cherry3.jpg',
    'image_id': 2,
    'probs': [0.007896028459072113,
     0.005379962734878063,
     0.08903055638074875,
     0.0013411621330305934,
     0.8963522911071777],
    'probs_sorted': {'values': [0.8963522911071777,
      0.08903055638074875,
      0.007896028459072113,
      0.005379962734878063,
      0.0013411621330305934],
     'indices': [4, 2, 0, 1, 3]}},